In [13]:
import pandas as pd
import numpy as np  # noqa
import optuna
import time
import logging  # noqa
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from lightgbm import LGBMRegressor, early_stopping, log_evaluation

# Enable logging
optuna.logging.set_verbosity(optuna.logging.INFO)


class TimingCallback:
    def __init__(self):
        self.start_time = None

    def __call__(self, study, trial):
        if self.start_time is None:
            self.start_time = time.time()
        else:
            elapsed_time = time.time() - self.start_time
            print(f"Trial {trial.number} finished in {elapsed_time:.2f} seconds.")
            self.start_time = time.time()

In [14]:
# Load data
data_dir = "/data3/lsf/Pein/Power-Prediction/data/"
file_name_common = 'farm_92'
train_data_selected = pd.read_csv(data_dir + f"train_{file_name_common}.csv")
test_data_selected = pd.read_csv(data_dir + f"test_{file_name_common}.csv")

# Define features and target
features = [
    col
    for col in train_data_selected.columns
    if col not in ["time", "lead_hour", "power"]
]
X_train = train_data_selected[features]
y_train = train_data_selected["power"]
X_test = test_data_selected[features]
y_test = test_data_selected["power"]

In [15]:
print(f'Shape of X_train: {X_train.shape}, Shape of y_train: {y_train.shape}')
print(f'Shape of X_test: {X_test.shape}, Shape of y_test: {y_test.shape}')

Shape of X_train: (14592, 89), Shape of y_train: (14592,)
Shape of X_test: (2880, 89), Shape of y_test: (2880,)


### Adding time features

In [16]:
# Convert 'time' column to datetime
train_data_selected["time"] = pd.to_datetime(train_data_selected["time"])
test_data_selected["time"] = pd.to_datetime(test_data_selected["time"])

In [17]:
# Function to add time features
def add_time_features(df):
    df["time"] = pd.to_datetime(df["time"])

    # Existing time features
    df["hour"] = df["time"].dt.hour
    df["quarter_hour"] = df["time"].dt.minute // 15

    # New time features
    df["day"] = df["time"].dt.day
    df["day_in_week"] = df["time"].dt.weekday

    # Sine and cosine transformations
    df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
    df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)

    df["quarter_hour_sin"] = np.sin(2 * np.pi * df["quarter_hour"] / 4)
    df["quarter_hour_cos"] = np.cos(2 * np.pi * df["quarter_hour"] / 4)

    df["day_sin"] = np.sin(2 * np.pi * df["day"] / 31)
    df["day_cos"] = np.cos(2 * np.pi * df["day"] / 31)

    df["day_in_week_sin"] = np.sin(2 * np.pi * df["day_in_week"] / 7)
    df["day_in_week_cos"] = np.cos(2 * np.pi * df["day_in_week"] / 7)

    return df

In [18]:
# # Add time features to both train and test data
train_data_selected = add_time_features(train_data_selected)
test_data_selected = add_time_features(test_data_selected)

In [19]:
# Define features including the new time features
time_features = [
    "hour",
    "quarter_hour",
    "day",
    "day_in_week",
    "hour_sin",
    "hour_cos",
    "quarter_hour_sin",
    "quarter_hour_cos",
    "day_sin",
    "day_cos",
    "day_in_week_sin",
    "day_in_week_cos",
]

In [20]:
all_features = features  + time_features

X_train = train_data_selected[all_features]
X_test = test_data_selected[all_features]

In [21]:
def objective(
    trial,
    X_train,
    y_train,
    X_test=None,
    y_test=None,
    use_test_for_validation_flag=False,
):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "boosting_type": "gbdt",
        "num_leaves": trial.suggest_int("num_leaves", 10, 80),
        "max_depth": trial.suggest_int("max_depth", 10, 50),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "n_estimators": trial.suggest_int("n_estimators", 500, 1500),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 50),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "num_threads": 16,
        "seed": 42,
    }

    model = LGBMRegressor(**params)

    if use_test_for_validation_flag and X_test is not None and y_test is not None:
        X_val_split, y_val_split = X_test, y_test
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_val_split, y_val_split)],
            eval_metric="rmse",
            callbacks=[early_stopping(stopping_rounds=15), log_evaluation(period=500)],
        )
        preds = model.predict(X_val_split)
    else:
        # Calculate the split index
        split_index = int(0.8 * len(X_train))

        # Split the data
        X_train_split, X_val_split = X_train[:split_index], X_train[split_index:]
        y_train_split, y_val_split = y_train[:split_index], y_train[split_index:]
        
        model.fit(
            X_train_split,
            y_train_split,
            eval_set=[(X_val_split, y_val_split)],
            eval_metric="rmse",
            callbacks=[early_stopping(stopping_rounds=100), log_evaluation(period=500)],
        )
        preds = model.predict(X_val_split)

    # Calculate RMSE
    rmse = root_mean_squared_error(y_val_split, preds)
    return rmse

In [22]:
# Create the study and optimize
use_test_for_validation_flag = False
study = optuna.create_study(direction="minimize")
study.optimize(
    lambda trial: objective(
        trial,
        X_train,
        y_train,
        X_test,
        y_test,
        use_test_for_validation_flag=use_test_for_validation_flag,
    ),
    n_trials=30,
    callbacks=[TimingCallback()],
)

# Get the best parameters
best_params = study.best_params
print("Best parameters found: ", best_params)

[I 2024-08-07 14:20:18,239] A new study created in memory with name: no-name-2f35a737-4a6f-4d14-a18c-01581519b1c8


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005700 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22878
[LightGBM] [Info] Number of data points in the train set: 11673, number of used features: 101
[LightGBM] [Info] Start training from score 564.183276
Training until validation scores don't improve for 100 rounds


[I 2024-08-07 14:20:18,581] Trial 0 finished with value: 361.3208677782303 and parameters: {'num_leaves': 28, 'max_depth': 15, 'learning_rate': 0.16552636426472944, 'n_estimators': 808, 'min_child_samples': 43, 'subsample': 0.5659593422415234, 'colsample_bytree': 0.8550790981732623}. Best is trial 0 with value: 361.3208677782303.


Early stopping, best iteration is:
[18]	valid_0's rmse: 361.321
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002519 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22878
[LightGBM] [Info] Number of data points in the train set: 11673, number of used features: 101
[LightGBM] [Info] Start training from score 564.183276
Training until validation scores don't improve for 100 rounds


[I 2024-08-07 14:20:18,991] Trial 1 finished with value: 359.613092031948 and parameters: {'num_leaves': 38, 'max_depth': 15, 'learning_rate': 0.17991063786833297, 'n_estimators': 1262, 'min_child_samples': 34, 'subsample': 0.7348078640655231, 'colsample_bytree': 0.8873377717742184}. Best is trial 1 with value: 359.613092031948.


...

Early stopping, best iteration is:
[25]	valid_0's rmse: 354.571
Trial 27 finished in 0.33 seconds.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002726 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22878
[LightGBM] [Info] Number of data points in the train set: 11673, number of used features: 101
[LightGBM] [Info] Start training from score 564.183276
Training until validation scores don't improve for 100 rounds


[I 2024-08-07 14:20:38,775] Trial 28 finished with value: 350.73729426434807 and parameters: {'num_leaves': 20, 'max_depth': 19, 'learning_rate': 0.09281879787338747, 'n_estimators': 1155, 'min_child_samples': 37, 'subsample': 0.6122687197116748, 'colsample_bytree': 0.7966872430791309}. Best is trial 25 with value: 341.7043715533523.


Early stopping, best iteration is:
[52]	valid_0's rmse: 350.737
Trial 28 finished in 0.37 seconds.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002827 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22878
[LightGBM] [Info] Number of data points in the train set: 11673, number of used features: 101
[LightGBM] [Info] Start training from score 564.183276
Training until validation scores don't improve for 100 rounds


[I 2024-08-07 14:20:39,167] Trial 29 finished with value: 370.86154589971875 and parameters: {'num_leaves': 28, 'max_depth': 13, 'learning_rate': 0.1614890618665271, 'n_estimators': 1323, 'min_child_samples': 43, 'subsample': 0.5365135472026565, 'colsample_bytree': 0.7897437546217961}. Best is trial 25 with value: 341.7043715533523.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iteration is:
[35]	valid_0's rmse: 370.862
Trial 29 finished in 0.39 seconds.
Best parameters found:  {'num_leaves': 21, 'max_depth': 21, 'learning_rate': 0.1170591971593041, 'n_estimators': 1350, 'min_child_samples': 35, 'subsample': 0.6506927490892368, 'colsample_bytree': 0.8103864882602075}


In [23]:
# best_params = {
#     "num_leaves": 70,
#     "max_depth": 36,
#     "learning_rate": 0.09304350950671668,
#     "n_estimators": 1158,
#     "min_child_samples": 18,
#     "subsample": 0.579731306036922,
#     "colsample_bytree": 0.8511910376678277,
# }

In [24]:
print("Best parameters found: ", best_params)
best_params["num_threads"] = 16

# Train the model with the best parameters
best_model = LGBMRegressor(**best_params)
best_model.fit(X_train, y_train)


# Extract feature importances
feature_importances = best_model.feature_importances_

# Create a DataFrame to hold the features and their importances
importance_df = pd.DataFrame(
    {"Feature": X_train.columns, "Importance": feature_importances}
)

# Sort the DataFrame by importance
importance_df = importance_df.sort_values(by="Importance", ascending=False)

Best parameters found:  {'num_leaves': 21, 'max_depth': 21, 'learning_rate': 0.1170591971593041, 'n_estimators': 1350, 'min_child_samples': 35, 'subsample': 0.6506927490892368, 'colsample_bytree': 0.8103864882602075}


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002841 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22879
[LightGBM] [Info] Number of data points in the train set: 14592, number of used features: 101
[LightGBM] [Info] Start training from score 540.706096


In [25]:
# Predict on the test set
y_pred = best_model.predict(X_test)

# Calculate RMSE
rmse = root_mean_squared_error(y_test, y_pred)
print(f"RMSE on the test set: {rmse:.4f}")

RMSE on the test set: 335.8639


In [26]:
# save important_features to csv file
if use_test_for_validation_flag:
    importance_df.to_csv("farm_important_features_use_test_for_validation.csv", index=False)
else:
    importance_df.to_csv("farm_important_features.csv", index=False)

In [27]:
# Set the threshold for feature importance
importance_threshold = 54

# Select features with importance greater than or equal to the threshold
selected_features = importance_df[importance_df["Importance"] >= importance_threshold][
    "Feature"
].tolist()

# Separate selected features into original input features and time features
selected_input_features = [
    feature for feature in selected_features if feature not in time_features
]
selected_time_features = [
    feature for feature in selected_features if feature in time_features
]

In [28]:
# Combine the filtered features with the power column and time features
train_data_final = pd.concat(
    [
        train_data_selected["time"],
        X_train[selected_input_features],
        train_data_selected["power"],
        train_data_selected[time_features],
    ],
    axis=1,
)
test_data_final = pd.concat(
    [
        test_data_selected["time"],
        X_test[selected_input_features],
        test_data_selected["power"],
        test_data_selected[time_features],
    ],
    axis=1,
)

# Ensure the final order of columns
final_columns = ["time"] + selected_input_features + ["power"] + time_features
train_data_final = train_data_final[final_columns]
test_data_final = test_data_final[final_columns]

# Display the new shapes of train_data_final and test_data_final
print("Shape of train_data_final:", train_data_final.shape)
print("Shape of test_data_final:", test_data_final.shape)
print("Selected input features:", selected_input_features)
print(f"Train columns : {train_data_final.columns}")

Shape of train_data_final: (14592, 98)
Shape of test_data_final: (2880, 98)
Selected input features: ['mgws', 'lgws', 'altitude', 'trpp', 'azimuth', 'deg0l', 'mudlp', 'blh', 'viwvn', 'kx', 'degm10l', 'mcc', 'gwd', 'p3020', 'totalx', 'lcc', 'mld', 'tcsw', 'sund', 'vimd', 'capes', 'bld', 'tcc', 'fg10', 'ishf', 'lspf', 'viwve', 'ewss', 'litoti', 'flsr', 'nsss', 'ilspf', 'str', 'u100', 'cape', 'hcc', 'u200', 'mlcape100', 'ws10', 'hwbt0', 'u10', 'ws200', 'msl', 'ws100', 'i10fg', 'ttrc', 'dsrp', 'mlcape50', 'u10n', 'slhf', 'hwbt1', 'ttr', 'mucape', 'fdir', 'cdir', 'strc', 'sshf', 'v200', 'v10n', 'ssrd', 'strd', 'd2m', 'v10', 'tcw', 'par', 'v100', 'uvb', 'tsr', 'tisr', 'sp', 'parcs', 't2m', 'mx2t', 'ssrc', 'tsrc', 'ssr', 'tcwv', 'mn2t', 'ssrdc', 'sst', 'skt', 'strdc', 'lblt', 'stl1']
Train columns : Index(['time', 'mgws', 'lgws', 'altitude', 'trpp', 'azimuth', 'deg0l', 'mudlp',
       'blh', 'viwvn', 'kx', 'degm10l', 'mcc', 'gwd', 'p3020', 'totalx', 'lcc',
       'mld', 'tcsw', 'sund', 'vimd'

In [29]:
# Save the final datasets to CSV
file_name_common = 'farm_98_withTime'
train_data_final.to_csv(data_dir + f"train_{file_name_common}.csv", index=False)
test_data_final.to_csv(data_dir + f"test_{file_name_common}.csv", index=False)
print("Final datasets saved to CSV.")

Final datasets saved to CSV.
